In [6]:
# Notebook: Tổng hợp số liệu Benign vs Malicious từ 2 Dataset

## 1. Import thư viện
import pandas as pd
import os

## 2. Danh sách các file dataset cần load
# Ví dụ: ['cic-ids2018.csv', 'unsw-nb15_example.xlsx']
file_paths = [
    '../dataset/unsw-nb15/UNSW_NB15_testing-set.csv',
    '../dataset/unsw-nb15/UNSW_NB15_training-set.csv',
]

## 3. Hàm load dataset theo định dạng

def load_dataset(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == '.csv':
        df = pd.read_csv(path)
    elif ext in ['.xls', '.xlsx']:
        df = pd.read_excel(path)
    else:
        raise ValueError(f"Không hỗ trợ định dạng file: {ext}")
    # đánh dấu tên dataset để phân biệt
    df['__dataset__'] = os.path.basename(path)
    return df

## 4. Đọc và gộp dữ liệu từ tất cả file
df_list = [load_dataset(p) for p in file_paths]
df_all = pd.concat(df_list, ignore_index=True)
print(f"Tổng số bản ghi gộp: {len(df_all)}")

## 5. Xác định cột nhãn chung
# Giả sử các file đều dùng cùng tên cột nhãn
if 'Label' in df_all.columns:
    label_col = 'Label'
elif 'label' in df_all.columns:
    label_col = 'label'
else:
    raise KeyError("Không tìm thấy cột nhãn ('Label' hoặc 'label') trong các DataFrame.")

## 6. Tính tổng summary gộp
# Định nghĩa mask Benign chung (string hoặc numeric)
if df_all[label_col].dtype == object:
    benign_mask = df_all[label_col] == 'Benign'
else:
    benign_mask = df_all[label_col] == 0

total_all = len(df_all)
benign_all = benign_mask.sum()
malicious_all = total_all - benign_all

print("=== Tổng hợp gộp 2 dataset ===")
print(f"Total: {total_all}")
print(f"Benign: {benign_all} ({benign_all/total_all*100:.2f}%)")
print(f"Malicious: {malicious_all} ({malicious_all/total_all*100:.2f}%)\n")

## 7. Chi tiết Breakdown theo loại tấn công (ở gộp)
# Nếu có cột 'attack_cat' thì ưu tiên breakdown theo cột này
if 'attack_cat' in df_all.columns:
    breakdown = df_all.loc[~benign_mask, 'attack_cat'].value_counts()
elif df_all[label_col].dtype == object:
    breakdown = df_all.loc[~benign_mask, label_col].value_counts()
else:
    breakdown = df_all.loc[~benign_mask, label_col].value_counts()

detail_all = pd.DataFrame({
    'Count': breakdown,
    'Percentage (%)': (breakdown/total_all*100).round(2)
}).sort_values('Count', ascending=False)

print("--- Breakdown gộp theo loại tấn công ---")
print(detail_all)

Tổng số bản ghi gộp: 257673
=== Tổng hợp gộp 2 dataset ===
Total: 257673
Benign: 93000 (36.09%)
Malicious: 164673 (63.91%)

--- Breakdown gộp theo loại tấn công ---
                Count  Percentage (%)
attack_cat                           
Generic         58871           22.85
Exploits        44525           17.28
Fuzzers         24246            9.41
DoS             16353            6.35
Reconnaissance  13987            5.43
Analysis         2677            1.04
Backdoor         2329            0.90
Shellcode        1511            0.59
Worms             174            0.07
